<a href="https://colab.research.google.com/github/saetta80/GenerativeAI/blob/main/Llama2_Sentiment_Analysis_Auto_response_generation_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Installation for GPU llama-cpp-python
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python --force-reinstall --upgrade --no-cache-dir --verbose

Using pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 82.6 MB/s eta 0:00:00
  Running command pip subprocess to install build dependencies
  Using pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
    Using cached scikit_build_core-0.7.0-py3-none-any.whl (136 kB)
    Using cached exceptiongroup-1.2.0-py3-none-any.whl (16 kB)
    Using cached packaging-23.2-py3-none-any.whl (53 kB)
    Using cached tomli-2.0.1-py3-none-any.whl (12 kB)
    Using cached pathspec-0.11.2-py3-none-any.whl (29 kB)
    Using cached pyproject_metadata-0.7.1-py3-none-any.whl (7.4 kB)
  ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
  lida 0.0.10 requires fastapi, which is not installed.
  lida 0.0.10 requires kaleido, which is not installed.
  lida 0.0.10 requires python-multipart,

In [2]:
# For downloading the models from HF Hub
!pip install huggingface_hub --force-reinstall --upgrade --verbose

Using pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
  Using cached huggingface_hub-0.19.4-py3-none-any.whl (311 kB)
  Using cached filelock-3.13.1-py3-none-any.whl (11 kB)
  Using cached fsspec-2023.12.1-py3-none-any.whl (168 kB)
  Using cached requests-2.31.0-py3-none-any.whl (62 kB)
  Using cached tqdm-4.66.1-py3-none-any.whl (78 kB)
  Using cached PyYAML-6.0.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (705 kB)
  Using cached typing_extensions-4.8.0-py3-none-any.whl (31 kB)
  Using cached packaging-23.2-py3-none-any.whl (53 kB)
  Using cached charset_normalizer-3.3.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (142 kB)
  Using cached idna-3.6-py3-none-any.whl (61 kB)
  Using cached urllib3-2.1.0-py3-none-any.whl (104 kB)
  Using cached certifi-2023.11.17-py3-none-any.whl (162 kB)
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.1.0
    Uninstalling urllib3-2.1.0:
      Removing file or directory /us

In [3]:
# For downloading datasets
!pip install datasets

  Using cached fsspec-2023.10.0-py3-none-any.whl (166 kB)
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2023.12.1
    Uninstalling fsspec-2023.12.1:
      Successfully uninstalled fsspec-2023.12.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2023.6.0 requires fsspec==2023.6.0, but you have fsspec 2023.10.0 which is incompatible.


In [4]:
# Import the hf_hub_download function from the Hugging Face Hub library
from huggingface_hub import hf_hub_download

# Import the Llama class from the llama_cpp library
from llama_cpp import Llama

In [5]:
model_name_or_path = "TheBloke/Llama-2-13B-chat-GGUF"
model_basename = "llama-2-13b-chat.Q5_K_M.gguf" # the model is in gguf format

In [6]:
model_path = hf_hub_download(
    repo_id=model_name_or_path,
    filename=model_basename
)

In [7]:
lcpp_llm = Llama(
    model_path=model_path,
    n_threads=2, # CPU cores
    n_batch=512, # 512 - Should be between 1 and n_ctx, consider the amount of VRAM in your GPU.
    n_gpu_layers=43, # 43 - Change this value based on your model and your GPU VRAM pool.
    n_ctx=4096, # Context window
)

AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | 


In [8]:
# Import the random module for generating random numbers or choices.
import random

# Import the pandas library and alias it as 'pd' for easier use.
import pandas as pd

# Import the 'load_dataset' function from the 'datasets' library for loading datasets.
from datasets import load_dataset

In [9]:
# Load the IMDb dataset using the 'load_dataset' function
dataset = load_dataset("imdb")

In [10]:
imdb_train_df = dataset['train'].to_pandas()   ## Tto_pandas is used to convert the training data into a pandas DataFrame.
imdb_test_df = dataset['test'].to_pandas()  ##: It converts the testing data into another pandas DataFrame.

In [11]:
(imdb_train_df.shape, imdb_test_df.shape)

((25000, 2), (25000, 2))

In [12]:
positive_examples = imdb_train_df.loc[imdb_train_df.label == 1, :].sample(3)
negative_examples = imdb_train_df.loc[imdb_train_df.label == 0, :].sample(3)

In [13]:
positive_examples

,text,label
17265,For anyone who wishes to get an impression of ...,1
14881,It was such a treat when this show was on beca...,1
18588,I claim no matter how hard I seek I'll never f...,1


In [14]:
negative_examples

,text,label
149,"This film was choppy, incoherent and contrived...",0
4094,I'm writing this because I somehow felt being ...,0
2884,This is the biggest piece of crap ever. It loo...,0


In [15]:
#concatenating the subsets of positive and negative sentiment examples (positive_examples and negative_examples)
examples = pd.concat([positive_examples, negative_examples]).to_json(orient='records')

#After concatenating the DataFrames, the to_json() method is applied to convert the combined DataFrame into a JSON (JavaScript Object Notation) format.
#The orient='records' argument specifies the format in which the JSON data should be structured.

In [16]:
examples

'[{"text":"For anyone who wishes to get an impression of the Soviet view of modern Russian history this monumental film is a treasure. The story starts at the turn of the century (1900) in the yellowish sepia colours of old photographs which improves to black and white during the middle of the century and to full colour when the story approaches modern times (i.e. the 1960s).<br \\/><br \\/>The story focuses on a boy in a remote Siberian village, who is marked by the arrival and arrest of an anarchist during the czarist era. He later joins the Bolsheviks after the revolution and brings soviet communism to his village. His son, by the local beauty, fights the Germans during the Second World War. When he returns to the village, the oil industry takes off and we are treated to some Soviet economic idealism.<br \\/><br \\/>This film is long and slow, but utterly logic and very well made. It can be seen in three parts.","label":1},{"text":"It was such a treat when this show was on because i

In [17]:
# Import the 'json' module for handling JSON data.
import json

# Import the 'numpy' library and alias it as 'np' for convenience.
import numpy as np

# Import the 'Counter' class from the 'collections' module for counting occurrences of elements.
from collections import Counter

# Import 'tqdm' for displaying progress bars when iterating over data.
from tqdm import tqdm


In [18]:
system_message = """[INST]<<SYS>>Classify the sentiment of movie reviews presented in the input as 'positive' or 'negative'.
Movie reviews will be delimited by triple backticks in the input.
Answer only 'positive' or 'negative'. Do not explain your answer.

Instructions:
1. Carefully read the text of the review and consider the overall sentiment of the review
2. Estimate the probability of the review being positive

To re-iterate, your answer should strictly only contain the label: positive or negative.

Some examples of expected output are provided below as guidance.<</SYS>>[/INST]
"""

In [19]:
prompt_template = """
[INST] ```{input_data}``` [/INST]
{output}
"""

In [20]:
## Initialize an empty string to store few-shot examples
few_shot_examples = ''

In [21]:
## Iterate through each example in the JSON data which was created earlier
for example in json.loads(examples):
        # Extract the input data (text) from the example, excluding the 'label'

    example_input = {i:example[i] for i in example if i!='label'}
    # Determine the sentiment prediction based on the 'label' value
    if example['label'] == 0:
        example_prediction = 'negative'
    else:
        example_prediction = 'positive'

    # Concatenate the input data and the predicted sentiment
    # using a template and add it to the 'few_shot_examples' string

    few_shot_examples += prompt_template.format(
        input_data=example_input['text'],  ###input_data is used in the prompt_template
        output=example_prediction          ###outpu is used in the prompt_template
    )

In [22]:
test_rows = json.loads(
    imdb_test_df.sample(100).to_json(orient='records')
)

In [23]:
## Initialize empty lists to store model predictions and ground truth values.
model_predictions, ground_truths = [], []

In [27]:
## Iterate through each row in the test data with a progress bar
for row in tqdm(test_rows):
      # Extract the input data (text) from the current row, excluding the 'label'
    test_input = {i:row[i] for i in row if i!='label'}

        # Construct a few-shot prompt by combining system message, few-shot examples, and test input
    few_shot_prompt = (
        system_message + few_shot_examples +
        prompt_template.format(
            input_data=test_input['text'],
            output=''
        )
    )

    # print(few_shot_prompt)

    try:
        # Use the model (lcpp_llm) to generate a response based on the few-shot prompt
        response = lcpp_llm(
            prompt=few_shot_prompt,
            max_tokens=2,
            temperature=0,
            top_p=0.95,
            repeat_penalty=1.2,
            top_k=50,
            stop=['INST'], # Dynamic stopping when such token is detected.
            echo=False # do not return the prompt
        )
        # Extract the model's prediction from the response

        prediction = response["choices"][0]["text"]

        # Append the model's prediction to the 'model_predictions' list, lowercased and stripped of whitespace
        model_predictions.append(prediction.strip().lower())

        # Determine the ground truth label based on the row's 'label' value and append it to 'ground_truths'
        if row['label'] == 0:
            ground_truths.append('negative')
        else:
            ground_truths.append('positive')
    except ValueError as e:
          # Handle any ValueErrors that may occur during the process and continue with the next row

        print(e)
        continue

100%|██████████| 100/100 [02:32<00:00,  1.52s/it]


In [28]:
Counter(model_predictions)

Counter({'negative': 43, 'positive': 55, 'the sentiment': 2})

In [29]:
Counter(ground_truths)

Counter({'negative': 46, 'positive': 54})

In [30]:
ground_truths = np.array(ground_truths)
model_predictions = np.array(model_predictions)


#ground_truths contains the true labels (ground truth) for a set of examples.
#model_predictions contains the labels predicted by a machine learning model for the same set of examples.

In [31]:
(ground_truths == model_predictions).mean()

0.93

In [32]:
TP = ((model_predictions == 'positive') & (ground_truths == 'positive')).sum()
FP = ((model_predictions == 'positive') & (ground_truths == 'negative')).sum()
precision = TP / (TP+FP)

In [33]:
precision

0.9272727272727272

In [34]:
dataset = load_dataset("AdiOO7/Bank_Complaints")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [35]:
system_message = """[INST]<<SYS>>As a spokesperson person of a particular bank, you are tasked to give a public response to a user's complaint presented as input.
Instructions:
1. Carefully observe the intensity and severity of the complaint received as input.
2. Choose a carefully worded public response. You need to reply to every complaint, however, responding with "Company chooses not to provide a public response" is also a valid response.
Some examples of appropriate response are provided below as guidance.<</SYS>>[/INST]
"""

In [36]:
prompt_template = """
[INST] {input_example} [/INST]
{output_example}
"""

In [37]:
few_shot_examples = ''

In [38]:
for i in range(5):
    sample_document = dataset['train'][random.randint(0, 1829)]
    user_input_example = sample_document['Input']
    assistant_output_example = sample_document['Response']

    few_shot_examples += prompt_template.format(
        input_example=user_input_example,
        output_example=assistant_output_example
    )

In [39]:
test_document = dataset['train'][random.randint(0, 1829)]
new_complaint = test_document['Input']

In [40]:
few_shot_prompt = (
    system_message +
    few_shot_examples +
    prompt_template.format(
        input_example=new_complaint,
        output_example=''
    )
)

In [41]:
response = lcpp_llm(
    prompt=few_shot_prompt,
    max_tokens=256,
    temperature=0,
    top_p=0.95,
    repeat_penalty=1.2,
    top_k=50,
    stop=['INST'], # Dynamic stopping when such token is detected.
    echo=False # do not return the prompt
)

Llama.generate: prefix-match hit


In [42]:
new_complaint

'Bank of America reduced my credit card limit, damaging my credit while I paid my out my amount in full and being a good payment \n'

In [43]:
print(response["choices"][0]["text"])

Company believes it acted appropriately as authorized by contract or law. However, we value our customers' satisfaction and would like to discuss this matter further with you. Please contact us at your earliest convenience so that we can work towards resolving the issue amicably.
